# Get Point-in-time datasets
Find all of the dates when companies first published their quarterly reports

In [2]:
import bql
import pandas as pd
import utils.dataRequestHelper as helper
import boto3
import json
from s3fs import S3FileSystem
import os

from ipywidgets import IntProgress
from IPython.display import display

In [3]:
import importlib
importlib.reload(helper)

<module 'utils.dataRequestHelper' from '/project/utils/dataRequestHelper.py'>

In [4]:
bq = bql.Service()

In [12]:
# Index to use for point in time firms
index = 'INDU Index'
filename = 'data_quarterly_pit_indu_refresh_blended.json'
reporting_period = 'Q'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]


# Calculate the Rebalance dates

For each rebalance date, get the members of the index at that time and look up their reporting dates

In [13]:
#all_data = []

def get_reporting_dates_per_rebalance(date, index):
    univ = bq.univ.members(index, dates=date)
    field = bq.data.sales_rev_turn(dates=bq.func.range('-5Y','0D'), fa_period_type=reporting_period) # Change to Annual if needed
    req = bql.Request(univ, field)
    data = bq.execute(req)
    df = data[0].df().dropna()
    return df.sort_values('PERIOD_END_DATE', ascending=True).reset_index().drop_duplicates(subset=['ID','PERIOD_END_DATE'], keep='first')

In [14]:
def get_rebalance_dates(index, rebalance_dates):
    all_data = []
    for date in rebalance_dates:
        all_data.append(get_reporting_dates_per_rebalance(date, index))
        print("Complete for ", date)
    df = pd.concat(all_data)
    df_concat = df[['ID','AS_OF_DATE','PERIOD_END_DATE']].sort_values('PERIOD_END_DATE', ascending=True).drop_duplicates(subset=['ID','PERIOD_END_DATE'], keep='first')
    return df_concat.set_index(['AS_OF_DATE','ID']).sort_values(['AS_OF_DATE'])

In [15]:
df_rebalance_dates = get_rebalance_dates(index, rebalance_dates)

Complete for  2024-12-31
Complete for  2024-09-30
Complete for  2024-06-30
Complete for  2024-03-31
Complete for  2023-12-31
Complete for  2023-09-30
Complete for  2023-06-30
Complete for  2023-03-31
Complete for  2022-12-31
Complete for  2022-09-30
Complete for  2022-06-30
Complete for  2022-03-31
Complete for  2021-12-31
Complete for  2021-09-30
Complete for  2021-06-30
Complete for  2021-03-31
Complete for  2020-12-31
Complete for  2020-09-30
Complete for  2020-06-30
Complete for  2020-03-31
Complete for  2019-12-31
Complete for  2019-09-30
Complete for  2019-06-30
Complete for  2019-03-31


## Request all of the data and format

Request financial statement and price points for each of the dates and securities. 

In [16]:
# Convert the request into correct format for data frame
def format_request_to_df(data, fields, index):
    fields = list(fields.keys())
    df_all = [data[index].df()[data[index].df()['PERIOD_END_DATE'] != 0]
                  .pivot(columns='PERIOD_END_DATE', values=[fields[index]])
                  .fillna(0) 
                  for index in range(0,len(fields))]
    df2 = pd.concat(df_all, axis=1)
    df3 = df2.stack().transpose().stack().unstack(level=0).transpose().fillna(0)
    df4 = df3.loc[:, (df3 != 0).any(axis=0)]
    # Reformat the columns to remove dates
    if len(df4.columns) == 6:
        df5 = df4.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
    else:
        df5 = df4.drop(columns=df4.columns[0:(len(df4.columns)-6)])
        df5 = df5.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
    # Reverse the direction of the dataset
    df6 = df5[df5.columns[::-1]]
    return df6.loc[(df6!=0).any(axis=1)]

In [17]:
# Convert datasets to dictionary
def convert_to_dict(securities, df_is, df_bs, df_px):
    date = {}
    for security in securities:
        # Convert DF to JSON
        data = {}
        df_is_sec = df_is.loc[security].to_json()
        df_bs_sec = df_bs.loc[security].to_json()
        df_px_sec = df_px.loc[security].set_index('DATE')[['Price']].to_json()
        # Convert to string and store
        data['is'] = json.dumps(df_is_sec)
        data['bs'] = json.dumps(df_bs_sec)
        data['px'] = json.dumps(df_px_sec)
        date[security] = data
    return date


def process_single_date(securities, fields, index):
    req = bql.Request(securities, fields)
    data = bq.execute(req)
    if len(fields) > 1:
        return format_request_to_df(data, fields, index)
    else:
        return data[0].df()


# main function for requesting the datasets
def update_financial_data(dates_and_securities, index):
    all_data = {}
    is_first = True
    dates = dates_and_securities.reset_index()['AS_OF_DATE'].unique()
    max_count = len(dates)
    f = IntProgress(min=0, max=max_count) # instantiate the bar
    display(f)
    # Loop through each date and extract securities
    for date in dates:
        if is_first:
            is_first=False
        else:
            as_of_date = str(date)[0:10]
            securities = list(dates_and_securities.loc[as_of_date].reset_index()['ID'])
            univ, is_fields, bs_fields, price = helper.setup_request(securities, as_of_date) 
            try:
                df_is = process_single_date(univ, is_fields)
                df_bs = process_single_date(univ, bs_fields)
                df_px = process_single_date(univ, price)
                all_data[as_of_date] = convert_to_dict(securities, df_is, df_bs, df_px)
            except:
                print(as_of_date)
            f.value += 1
    return all_data
        

In [18]:
df_rebalance_dates.loc['2024-10-22']

PERIOD_END_DATE
AS_OF_DATE ID                           
2024-10-22 VZ UN Equity       2024-09-30
           MMM UN Equity      2024-09-30
           SHW UN Equity      2024-09-30
           RTX UN Equity      2024-09-30

In [19]:
# request the data
all_data = update_financial_data(df_rebalance_dates, index)

IntProgress(value=0, max=428)

2020-04-14
2020-04-15
2020-04-17
2020-04-20
2020-04-21
2020-04-23
2020-04-24
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-18
2020-05-19
2020-05-21
2020-05-28
2020-06-03
2020-06-25
2020-07-09
2020-07-16
2020-07-20
2020-07-21
2020-07-23
2020-07-24
2020-07-28


KeyboardInterrupt: 

## Save the PIT data to S3

Store this in the data bucket for the project to retrieve for the next step

In [55]:
## Save to S3
user_bucket_name = os.environ['BQUANT_SANDBOX_USER_BUCKET']
bqnt_username = os.environ['BQUANT_USERNAME']

#s3 = boto3.resource("s3")

path_to_s3 = f's3://{user_bucket_name}/{bqnt_username}/tmp/fs/{filename}'
s3 = S3FileSystem()

with s3.open(path_to_s3, 'w') as file:
     json.dump(all_data, file)

## Request datasets for Training
Use the S&P 500 and select random securities to generate the prompts that do not appear in the INDU Index. This will be used to train the data

In [4]:
# select the index
training_index = 'SPX Index'

filename = 'data_quarterly_pit_indu_refresh_blended.json'
reporting_period = 'Q'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]



In [ ]:
df_rebalance_dates = get_rebalance_dates(training_index)

### Review of files in S3

In [10]:
import utils.model_helper as mh
import importlib
importlib.reload(mh)

<module 'utils.model_helper' from '/project/utils/model_helper.py'>

In [11]:
helper = mh.ModelHelper('tmp/fs')

In [12]:
helper.delete_model_in_s3('qwen')

bclarke16/tmp/fs/20250208_Run1.json
bclarke16/tmp/fs/data.json
bclarke16/tmp/fs/data_annual_pit_indu.json
bclarke16/tmp/fs/data_annual_pit_spx.json
bclarke16/tmp/fs/data_quarterly.json
bclarke16/tmp/fs/data_quarterly_pit.json
bclarke16/tmp/fs/data_quarterly_pit_indu.json
bclarke16/tmp/fs/data_quarterly_pit_indu_refresh.json
bclarke16/tmp/fs/data_quarterly_pit_indu_refresh_blended.json
bclarke16/tmp/fs/data_quarterly_pit_indu_refresh_no_calendar.json
bclarke16/tmp/fs/data_quarterly_pit_indu_refresh_v2.json
bclarke16/tmp/fs/datapack/config/data_item_definitions.parquet
bclarke16/tmp/fs/datapack/config/data_pack_config.parquet
bclarke16/tmp/fs/datapack/config/data_pack_metadata.parquet
bclarke16/tmp/fs/datapack/config/universe_definitions.parquet
bclarke16/tmp/fs/datapack/data_item/analyst_rating.parquet
bclarke16/tmp/fs/datapack/data_item/beta.parquet
bclarke16/tmp/fs/datapack/data_item/bics_level_1.parquet
bclarke16/tmp/fs/datapack/data_item/bics_level_2.parquet
bclarke16/tmp/fs/datapac

In [56]:
all_data['2024-10-22']['MMM UN Equity']a

{'is': '"{\\"t\\":{\\"01 Revenue (Adj)\\":28565000000.0,\\"02 Sales and Services Revenues (Adj)\\":28565000000.0,\\"05 Cost of Revenue (Adj)\\":16225000000.0,\\"06 Cost of Goods & Services Sold (Adj)\\":16225000000.0,\\"08 Gross Profit (Adj)\\":12340000000.0,\\"10 Operating Expenses (Adj)\\":7008000000.0,\\"11 Selling, General and Administrative Expense (Adj)\\":5555000000.0,\\"12 R&D Expense Adjusted (Adj)\\":1453000000.0,\\"13 Other Operating Expenses (Adj)\\":0.0,\\"14 Operating Income or Losses (Adj)\\":5332000000.0,\\"15 Non-Operating (Income) Loss (Adj)\\":-36000000.0,\\"16 Net Interest Expense (Adj)\\":0.0,\\"17 Interest Expense (Adj)\\":1262000000.0,\\"18 Interest Income (Adj)\\":0.0,\\"20 Other Non-Operating (Income) Loss (Adj)\\":-938000000.0,\\"21 Pretax Income (Loss), Adjusted (Adj)\\":5368000000.0,\\"22 Abnormal Losses (Gains)\\":1447000000.0,\\"24 Sale of Business\\":0.0,\\"27 Other Abnormal Items\\":181000000.0,\\"28 Pretax Income (Loss), GAAP\\":5368000000.0,\\"29 Incom

## Unused Code

In [44]:
univ = ['MMM UN Equity', 'SHW UN Equity', 'VZ UN Equity', 'RTX UN Equity']
#univ, is_fields, bs_fields, price = helper.setup_request(univ, '2025-01-21') 
univ, is_fields, bs_fields, price = helper.setup_request(univ, '2024-10-22') 


In [45]:
req = bql.Request(univ, is_fields)
data = bq.execute(req)

In [46]:
fields = is_fields
fields = list(fields.keys())
df_all = [data[index].df()[data[index].df()['PERIOD_END_DATE'] != 0]
              .pivot(columns='PERIOD_END_DATE', values=[fields[index]])
              .fillna(0) 
              for index in range(0,len(fields))]
df2 = pd.concat(df_all, axis=1)
df3 = df2.stack().transpose().stack().unstack(level=0).transpose().fillna(0)
df4 = df3.loc[:, (df3 != 0).any(axis=0)]
# Reformat the columns to remove dates
# if len(df4.columns) == 6:
#      df5 = df4.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
# else:
#      df5 = df4.drop(columns=df4.columns[0:(len(df4.columns)-6)])
#     df5 = df5.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
# # Reverse the direction of the dataset
# df6 = df5[df5.columns[::-1]]

In [47]:
df4

PERIOD_END_DATE                                                 2023-06-30  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                3.101200e+10   
              02 Sales and Services Revenues (Adj)            3.101200e+10   
              03 Financing Revenue (Adj)                      0.000000e+00   
              04 Other Revenue (Adj)                          0.000000e+00   
              05 Cost of Revenue (Adj)                        1.765400e+10   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         4.796038e+00   
              48 Diluted Weighted Average Shares              4.208750e+09   
              49 Diluted EPS                                  5.000000e+00   
              50 Diluted EPS from Continuing Operations       5.000000e+00   
              51 Diluted EPS from Continuing Operations, Adj  4.787209e+00   

PERIOD_END_DATE                                                 2023-09-30  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                2.866300e+10   
              02 Sales and Services Revenues (Adj)            2.866300e+10   
              03 Financing Revenue (Adj)                      0.000000e+00   
              04 Other Revenue (Adj)                          0.000000e+00   
              05 Cost of Revenue (Adj)                        1.664200e+10   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         4.699173e+00   
              48 Diluted Weighted Average Shares              4.211750e+09   
              49 Diluted EPS                                  4.960000e+00   
              50 Diluted EPS from Continuing Operations       4.960000e+00   
              51 Diluted EPS from Continuing Operations, Adj  4.689995e+00   

PERIOD_END_DATE                                                 2023-12-31  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                3.268100e+10   
              02 Sales and Services Revenues (Adj)            3.268100e+10   
              03 Financing Revenue (Adj)                      0.000000e+00   
              04 Other Revenue (Adj)                          0.000000e+00   
              05 Cost of Revenue (Adj)                        1.847700e+10   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         4.567779e+00   
              48 Diluted Weighted Average Shares              4.213500e+09   
              49 Diluted EPS                                  2.760000e+00   
              50 Diluted EPS from Continuing Operations       2.760000e+00   
              51 Diluted EPS from Continuing Operations, Adj  4.562782e+00   

PERIOD_END_DATE                                                 2024-03-31  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                3.265300e+10   
              02 Sales and Services Revenues (Adj)            3.265300e+10   
              03 Financing Revenue (Adj)                      0.000000e+00   
              04 Other Revenue (Adj)                          0.000000e+00   
              05 Cost of Revenue (Adj)                        1.819300e+10   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         4.508337e+00   
              48 Diluted Weighted Average Shares              4.215500e+09   
              49 Diluted EPS                                  2.680000e+00   
              50 Diluted EPS from Continuing Operations       2.680000e+00   
              51 Di

In [13]:
df_is = process_single_date(univ, is_fields)
df_bs = process_single_date(univ, bs_fields)
df_px = process_single_date(univ, price)

ValueError: Length mismatch: Expected axis has 1 elements, new values have 6 elements

In [19]:
df_is

t  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                6.294000e+09   
              02 Sales and Services Revenues (Adj)            6.294000e+09   
              05 Cost of Revenue (Adj)                        3.647000e+09   
              06 Cost of Goods & Services Sold (Adj)          3.647000e+09   
              08 Gross Profit (Adj)                           2.647000e+09   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         1.159716e+00   
              48 Diluted Weighted Average Shares              4.225000e+09   
              49 Diluted EPS                                  7.800000e-01   
              50 Diluted EPS from Continuing Operations       7.800000e-01   
              51 Diluted EPS from Continuing Operations, Adj  1.155858e+00   

                                                                       t-1  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                6.255000e+09   
              02 Sales and Services Revenues (Adj)            6.255000e+09   
              05 Cost of Revenue (Adj)                        3.571000e+09   
              06 Cost of Goods & Services Sold (Adj)          3.571000e+09   
              08 Gross Profit (Adj)                           2.684000e+09   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         1.113879e+00   
              48 Diluted Weighted Average Shares              4.221000e+09   
              49 Diluted EPS                                  1.090000e+00   
              50 Diluted EPS from Continuing Operations       1.090000e+00   
              51 Diluted EPS from Continuing Operations, Adj  1.113879e+00   

                                                                       t-2  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                8.003000e+09   
              02 Sales and Services Revenues (Adj)            8.003000e+09   
              05 Cost of Revenue (Adj)                        4.329000e+09   
              06 Cost of Goods & Services Sold (Adj)          4.329000e+09   
              08 Gross Profit (Adj)                           3.674000e+09   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         1.110558e+00   
              48 Diluted Weighted Average Shares              4.219000e+09   
              49 Diluted EPS                                  1.090000e+00   
              50 Diluted EPS from Continuing Operations       1.090000e+00   
              51 Diluted EPS from Continuing Operations, Adj  1.108725e+00   

                                                                       t-3  \
ID                                                                           
MMM UN Equity 01 Revenue (Adj)                                8.013000e+09   
              02 Sales and Services Revenues (Adj)            8.013000e+09   
              05 Cost of Revenue (Adj)                        4.678000e+09   
              06 Cost of Goods & Services Sold (Adj)          4.678000e+09   
              08 Gross Profit (Adj)                           3.335000e+09   
...                                                                    ...   
VZ UN Equity  47 Basic EPS from Continuing Operations         1.037731e+00   
              48 Diluted Weighted Average Shares              4.214000e+09   
              49 Diluted EPS                                 -6.400000e-01   
              50 Diluted EPS from Continuing Operations      -6.400000e-01   
              51 Diluted EPS from Continuing Operations, Adj  1.037731e+00   

             

In [1]:

#convert_to_dict(['MMM UN Equity', 'SHW UN Equity', 'VZ UN Equity', 'RTX UN Equity'],df_is,df_bs,df_px)
#json.dumps(df_is.loc['MMM UN Equity'].to_json())

In [91]:
# Failed Dates
failed_dates = ['2020-08-17',
'2020-09-21',
'2021-08-16',
'2021-09-24',
'2022-09-28',
'2022-11-25',
'2023-10-25',
'2024-01-08',
'2024-06-17',
'2024-11-08',
'2025-01-08']

In [8]:
# # pull out the list of as of dates - will use this to loop through the securities
# dates = df_rebalance_dates.reset_index()['AS_OF_DATE'].unique()

In [1]:
# as_of_date = '2020-03-11'#str(dates[12])[0:10]
# securities = list(df_rebalance_dates.loc[as_of_date].index)
# as_of_date

In [73]:
# univ, is_fields, bs_fields, price = helper.setup_request(securities, as_of_date)

In [74]:
# req = bql.Request(securities,is_fields)
# data = bq.execute(req)

In [80]:
# # Convert the request into correct format for data frame
# def format_request_to_df(data, fields):
#     fields = list(fields.keys())
#     df_all = [data[index].df()[data[index].df()['PERIOD_END_DATE'] != 0]
#                   .pivot(columns='PERIOD_END_DATE', values=[fields[index]])
#                   .fillna(0) 
#                   for index in range(0,len(fields))]
#     df2 = pd.concat(df_all, axis=1)
#     df3 = df2.stack().transpose().stack().unstack(level=0).transpose().fillna(0)
#     df4 = df3.loc[:, (df3 != 0).any(axis=0)]
#     # Reformat the columns to remove dates
#     if len(df4.columns) == 6:
#         df5 = df4.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
#     else:
#         df5 = df4.drop(columns=df4.columns[0:(len(df4.columns)-6)])
#         df5 = df5.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
#     # Reverse the direction of the dataset
#     df6 = df5[df5.columns[::-1]]
#     return df6.loc[(df6!=0).any(axis=1)]

In [90]:
# # test run
# req = bql.Request(securities, is_fields)
# data_is = bq.execute(req)
# req = bql.Request(securities, bs_fields)
# data_bs = bq.execute(req)
# df_is = format_request_to_df(data_is, is_fields)
# df_bs = format_request_to_df(data_bs, bs_fields)

In [75]:
# fields = list(is_fields.keys())
# df_all = [data[index].df()[data[index].df()['PERIOD_END_DATE'] != 0]
#               .pivot(columns='PERIOD_END_DATE', values=[fields[index]])
#               .fillna(0) 
#               for index in range(0,len(fields))]
# df2 = pd.concat(df_all, axis=1)
# df3 = df2.stack().transpose().stack().unstack(level=0).transpose().fillna(0)
# df4 = df3.loc[:, (df3 != 0).any(axis=0)]